In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_columns=1000
pd.options.display.max_rows=1000
import numpy as np
import multiprocessing as mp
import time

In [2]:
def get_mode(x):
    try:
        return x.value_counts().index[0]
    except:
        return np.nan
    
def get_mode_count(x):
    try:
        return x.value_counts().values[0]
    except:
        return np.nan
    
def parse_dict_of_dict(_dict, _str = ''):
    ret_dict = {}
    for k, v in _dict.items():
        if isinstance(v, dict):
            ret_dict.update(parse_dict_of_dict(v, _str= '_'.join([_str, k]).strip('_')))
        elif isinstance(v, list):
            for index, item in enumerate(v):
                if isinstance(item, dict):
                    ret_dict.update(parse_dict_of_dict(item,  _str= '_'.join([_str, k, str(index)]).strip('_')))
                else:
                    ret_dict['_'.join([_str, k, str(index)]).strip('_')] = item
        else:
            try:
                ret_dict['_'.join([_str, k]).strip('_')] = str(v)
            except Exception as e:
                ret_dict['_'.join([_str, k]).strip('_')] = unicode.encode(v, errors='ignore')
    return ret_dict

In [3]:
DATA_DIR = ''

train = pd.read_csv(DATA_DIR + 'train.csv')
"train", train.shape

test = pd.read_csv(DATA_DIR + 'test_QyjYwdj.csv')
"test", test.shape

campaign_data = pd.read_csv(DATA_DIR + 'campaign_data.csv')
"campaign_data", campaign_data.shape
campaign_data['start_date'] = pd.to_datetime(campaign_data.start_date, format='%d/%m/%y')
campaign_data['end_date'] = pd.to_datetime(campaign_data.end_date,  format='%d/%m/%y')
campaign_data['duration'] = (campaign_data.end_date - campaign_data.start_date).dt.days 

customer_demographics = pd.read_csv(DATA_DIR + 'customer_demographics.csv')
"customer_demographics", customer_demographics.shape 

customer_transaction_data = pd.read_csv(DATA_DIR + 'customer_transaction_data.csv')
"customer_transaction_data", customer_transaction_data.shape 
customer_transaction_data = customer_transaction_data.drop_duplicates()
"customer_transaction_data", customer_transaction_data.shape 

customer_transaction_data['date'] = pd.to_datetime(customer_transaction_data.date, format='%Y-%m-%d')

item_data = pd.read_csv(DATA_DIR + 'item_data.csv')
"item_data",item_data.shape

coupon_item_mapping = pd.read_csv(DATA_DIR + 'coupon_item_mapping.csv')
"coupon_item_mapping", coupon_item_mapping.shape


('train', (78369, 5))

('test', (50226, 4))

('campaign_data', (28, 4))

('customer_demographics', (760, 7))

('customer_transaction_data', (1324566, 7))

('customer_transaction_data', (1321650, 7))

('item_data', (74066, 4))

('coupon_item_mapping', (92663, 2))

In [4]:
df_test = test.merge(campaign_data, how='left')
df_test.shape
df_test.head()

(50226, 8)

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration
0,3,22,869,967,X,2013-09-16,2013-10-18,32
1,4,20,389,1566,Y,2013-09-07,2013-11-16,70
2,5,22,981,510,X,2013-09-16,2013-10-18,32
3,8,25,1069,361,Y,2013-10-21,2013-11-22,32
4,10,17,498,811,Y,2013-07-29,2013-08-30,32


In [5]:
df_train = train.merge(campaign_data, how='left')
df_train.shape
df_train.head()

(78369, 9)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration
0,1,13,27,1053,0,X,2013-05-19,2013-07-05,47
1,2,13,116,48,0,X,2013-05-19,2013-07-05,47
2,6,9,635,205,0,Y,2013-03-11,2013-04-12,32
3,7,13,644,1050,0,X,2013-05-19,2013-07-05,47
4,9,8,1017,1489,0,X,2013-02-16,2013-04-05,48


In [6]:
customer_transaction_data.date.min(), customer_transaction_data.date.max()
df_train.start_date.min(), df_train.end_date.max()
df_test.start_date.min(), df_test.end_date.max()

(Timestamp('2012-01-02 00:00:00'), Timestamp('2013-07-03 00:00:00'))

(Timestamp('2012-08-12 00:00:00'), Timestamp('2013-07-05 00:00:00'))

(Timestamp('2013-07-15 00:00:00'), Timestamp('2013-12-20 00:00:00'))

In [7]:
df_all = pd.concat([df_train, df_test], sort=False, axis=0)
df_all.shape
df_all.head()

(128595, 9)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration
0,1,13,27,1053,0.0,X,2013-05-19,2013-07-05,47
1,2,13,116,48,0.0,X,2013-05-19,2013-07-05,47
2,6,9,635,205,0.0,Y,2013-03-11,2013-04-12,32
3,7,13,644,1050,0.0,X,2013-05-19,2013-07-05,47
4,9,8,1017,1489,0.0,X,2013-02-16,2013-04-05,48


In [8]:
cust_transactions = customer_transaction_data.merge(item_data, on = 'item_id', how='left')
cust_transactions['coupon_discount'] = cust_transactions.coupon_discount.abs()
cust_transactions['other_discount'] = cust_transactions.other_discount.abs()
cust_transactions['coupon_applied'] = (cust_transactions['coupon_discount'] > 0).astype(int)
cust_transactions.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,coupon_applied
0,2012-01-02,1501,26830,1,35.26,10.69,0.0,56,Local,Natural Products,0
1,2012-01-02,1501,54253,1,53.43,13.89,0.0,56,Local,Natural Products,0
2,2012-01-02,1501,31962,1,106.50,14.25,0.0,524,Established,Grocery,0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery,0
4,2012-01-02,1501,48199,1,71.24,28.14,0.0,524,Established,Grocery,0


In [9]:
df_ = df_all[['campaign_id', 'coupon_id', 'customer_id', 'redemption_status', 'campaign_type', 'start_date', 'end_date']].drop_duplicates()
df_.shape

(128573, 7)

In [10]:
df_x = df_all[['customer_id', 'start_date']].drop_duplicates().merge(cust_transactions, on=['customer_id'])
df_x = df_x[df_x.start_date > df_x.date]
df_x = df_x.sort_values(by=['customer_id', 'date'])

cust_hist_trans = df_x.groupby(['customer_id', 'start_date']).agg({
            'item_id': ['count'],
            'selling_price': ['sum', 'mean', 'std'],
            'other_discount': ['sum', 'mean'],
            'coupon_discount': ['sum', 'mean'], 
            'coupon_applied': ['sum', 'mean'], 
})

cust_hist_trans.columns = ['_'.join(col).strip('_') for col in cust_hist_trans.columns.values]
cust_hist_trans = cust_hist_trans.reset_index()
cust_hist_trans.shape
cust_hist_trans.head()

(6940, 12)

,customer_id,start_date,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean
0,1,2012-10-08,473,44606.33,94.305137,55.196674,8723.61,18.443150,644.69,1.362981,30,0.063425
1,1,2013-02-16,732,67753.78,92.559809,52.082711,12524.18,17.109536,1422.26,1.942978,54,0.073770
2,1,2013-04-22,918,85861.11,93.530621,52.064067,15703.24,17.105926,2045.59,2.228312,76,0.082789
3,1,2013-05-19,975,91076.19,93.411477,51.866427,16142.43,16.556338,2045.59,2.098041,76,0.077949
4,1,2013-08-10,1046,98165.57,93.848537,51.577051,16902.53,16.159207,2045.59,1.955631,76,0.072658


In [11]:
cust_hist_trans.to_csv('cust_hist_trans.csv', index=False)

In [12]:
cust_daily = cust_transactions.groupby(['date', 'customer_id']).agg({
    'item_id': 'count',
    'quantity': 'sum',
    'selling_price': 'sum',
    'other_discount': 'sum',
    'coupon_discount': 'sum',
    'coupon_applied': 'sum'
})

# cust_daily.columns = ['_'.join(col).strip('_') for col in cust_daily.columns.values]
cust_daily = cust_daily.reset_index()
cust_daily = cust_daily.sort_values(by=['customer_id', 'date'])

cust_daily['selling_price_ewm'] = cust_daily.groupby('customer_id')['selling_price'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()
cust_daily['coupon_discount_ewm'] = cust_daily.groupby('customer_id')['coupon_discount'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()
cust_daily['coupon_applied_ewm'] = cust_daily.groupby('customer_id')['coupon_applied'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()

# cust_daily.head()

df_x = df_all[['customer_id', 'start_date']].drop_duplicates().merge(cust_daily, on=['customer_id'])
df_x = df_x[df_x.start_date > df_x.date]
df_x = df_x.sort_values(by=['customer_id', 'start_date', 'date'])


cust_hist_trans_daily = df_x.groupby(['customer_id', 'start_date']).agg({
    'date': ['count'],
    'selling_price': ['mean', 'std', 'last'],
    'other_discount': ['mean', 'std', 'last'],
    'coupon_discount': ['mean', 'std', 'last'],
    'coupon_applied': ['mean', 'std', 'last'],
    'selling_price_ewm': ['mean', 'std'],
    'coupon_discount_ewm': ['mean', 'std'],
    'coupon_applied_ewm': ['mean', 'std'],
})

cust_hist_trans_daily.columns = ['_'.join(col).strip('_') for col in cust_hist_trans_daily.columns.values]
cust_hist_trans_daily = cust_hist_trans_daily.reset_index()
cust_hist_trans_daily.shape
cust_hist_trans_daily.head()

(6940, 21)

,customer_id,start_date,date_count,selling_price_mean,selling_price_std,selling_price_last,other_discount_mean,other_discount_std,other_discount_last,coupon_discount_mean,coupon_discount_std,coupon_discount_last,coupon_applied_mean,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std
0,1,2012-10-08,24,1858.597083,976.818611,398.23,363.483750,197.013421,98.67,26.862083,43.866142,0.00,1.250000,2.269840,0,1879.518842,645.998993,26.690546,29.446849,1.228160,1.505524
1,1,2013-02-16,35,1935.822286,961.359090,2808.97,357.833714,190.622169,442.76,40.636000,59.906843,204.82,1.542857,2.266675,6,1928.605526,667.519926,38.327243,45.608960,1.479199,1.565112
2,1,2013-04-22,43,1996.770000,981.487965,2282.17,365.191628,197.079516,269.65,47.571860,62.933230,138.91,1.767442,2.338469,6,1996.526399,684.839244,46.758275,49.871721,1.728138,1.675490
3,1,2013-05-19,48,1897.420625,1035.892776,142.12,336.300625,209.398916,35.26,42.616458,61.277457,0.00,1.583333,2.276927,0,1916.841850,723.749845,42.886346,48.673791,1.590956,1.643552
4,1,2013-08-10,56,1752.956607,1080.929520,266.08,301.830893,219.662424,12.46,36.528393,58.610462,0.00,1.357143,2.177810,0,1765.355731,797.854447,36.763262,47.471880,1.363828,1.619723


In [13]:
cust_hist_trans_daily.to_csv('cust_hist_trans_daily.csv', index=False)

In [14]:
coup_trans = coupon_item_mapping.merge(cust_transactions, on='item_id', how='left')
coup_trans.shape
coup_trans.head()

(2095717, 12)

,coupon_id,item_id,date,customer_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,coupon_applied
0,105,37,2012-03-11,1044.0,1.0,47.73,11.04,0.0,56.0,Local,Grocery,0.0
1,105,37,2012-05-15,1044.0,3.0,142.48,33.84,0.0,56.0,Local,Grocery,0.0
2,105,37,2012-08-23,1044.0,3.0,142.48,27.43,0.0,56.0,Local,Grocery,0.0
3,105,37,2012-10-09,1044.0,2.0,95.11,18.17,0.0,56.0,Local,Grocery,0.0
4,105,37,2012-10-26,1044.0,3.0,169.91,0.00,0.0,56.0,Local,Grocery,0.0


In [15]:
df_x = df_all[['coupon_id', 'start_date']].drop_duplicates().merge(coup_trans, on='coupon_id', how='left')
df_x = df_x[df_x.start_date > df_x.date]
df_x = df_x.sort_values(by=['coupon_id', 'date'])
df_x.shape

coup_hist_trans = df_x.groupby(['coupon_id', 'start_date']).agg({
            'item_id': ['count'],
            'selling_price': ['sum', 'mean', 'std'],
            'other_discount': ['sum', 'mean'],
            'coupon_discount': ['sum', 'mean'], 
            'coupon_applied': ['sum', 'mean'], 
})


coup_hist_trans.columns = ['_'.join(col).strip('_') for col in coup_hist_trans.columns.values]
coup_hist_trans = coup_hist_trans.reset_index()
coup_hist_trans.shape
coup_hist_trans.head()

(1968667, 13)

(1334, 12)

,coupon_id,start_date,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean
0,1,2012-08-12,85,11452.11,134.730706,67.816665,1556.18,18.308000,53.43,0.628588,3.0,0.035294
1,2,2012-08-12,25,2721.03,108.841200,38.436685,385.83,15.433200,0.00,0.000000,0.0,0.000000
2,3,2012-10-08,60,8620.75,143.679167,52.573078,509.73,8.495500,151.38,2.523000,4.0,0.066667
3,4,2012-11-19,385,69929.60,181.635325,39.466516,14835.73,38.534364,587.73,1.526571,31.0,0.080519
4,5,2012-11-19,31,11380.15,367.101613,148.904110,958.17,30.908710,0.00,0.000000,0.0,0.000000


In [16]:
coup_hist_trans.to_csv('coup_hist_trans.csv', index=False)

In [17]:
coup_daily = coup_trans.groupby(['coupon_id', 'date']).agg({
    'item_id': 'count',
    'quantity': 'sum',
    'selling_price': 'sum',
    'other_discount': 'sum',
    'coupon_discount': 'sum',
    'coupon_applied': 'sum'
})

# cust_daily.columns = ['_'.join(col).strip('_') for col in cust_daily.columns.values]
coup_daily = coup_daily.reset_index()
coup_daily = coup_daily.sort_values(by=['coupon_id', 'date'])

coup_daily['selling_price_ewm'] = coup_daily.groupby('coupon_id')['selling_price'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()
coup_daily['coupon_discount_ewm'] = coup_daily.groupby('coupon_id')['coupon_discount'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()
coup_daily['coupon_applied_ewm'] = coup_daily.groupby('coupon_id')['coupon_applied'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()


df_x = df_all[['coupon_id', 'start_date']].drop_duplicates().merge(coup_daily, on=['coupon_id'])
df_x = df_x[df_x.start_date > df_x.date]
df_x = df_x.sort_values(by=['coupon_id', 'start_date', 'date'])


coup_hist_trans_daily = df_x.groupby(['coupon_id', 'start_date']).agg({
    'date': ['count'],
    'selling_price': ['mean', 'std', 'last'],
    'other_discount': ['mean', 'std', 'last'],
    'coupon_discount': ['mean', 'std', 'last'],
    'coupon_applied': ['mean', 'std', 'last'],
    'selling_price_ewm': ['mean', 'std'],
    'coupon_discount_ewm': ['mean', 'std'],
    'coupon_applied_ewm': ['mean', 'std'],
})

coup_hist_trans_daily.columns = ['_'.join(col).strip('_') for col in coup_hist_trans_daily.columns.values]
coup_hist_trans_daily = coup_hist_trans_daily.reset_index()
coup_hist_trans_daily.shape
coup_hist_trans_daily.head()

(1334, 21)

,coupon_id,start_date,date_count,selling_price_mean,selling_price_std,selling_price_last,other_discount_mean,other_discount_std,other_discount_last,coupon_discount_mean,coupon_discount_std,coupon_discount_last,coupon_applied_mean,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std
0,1,2012-08-12,56,204.501964,137.281587,89.05,27.788929,70.372353,31.70,0.954107,4.046571,0.0,0.053571,0.227208,0.0,204.776877,94.739820,0.950181,2.780504,0.053351,0.156120
1,2,2012-08-12,25,108.841200,38.436685,89.05,15.433200,11.640063,21.02,0.000000,0.000000,0.0,0.000000,0.000000,0.0,108.763484,29.324725,0.000000,0.000000,0.000000,0.000000
2,3,2012-10-08,32,269.398438,182.429744,234.38,15.929063,34.059894,14.96,4.730625,16.307396,0.0,0.125000,0.421212,0.0,270.313425,136.649589,4.708867,13.273592,0.124400,0.346728
3,4,2012-11-19,183,382.128962,236.906274,710.60,81.069563,70.270201,356.20,3.211639,8.661850,0.0,0.169399,0.443194,0.0,380.691240,172.148009,3.179423,6.579928,0.167792,0.330903
4,5,2012-11-19,29,392.418966,202.856451,142.12,33.040345,44.833034,178.10,0.000000,0.000000,0.0,0.000000,0.000000,0.0,391.723653,149.206321,0.000000,0.000000,0.000000,0.000000


In [18]:
coup_hist_trans_daily.to_csv('coup_hist_trans_daily.csv', index=False)

In [19]:
df_x = df_.merge(coup_trans, on=['customer_id', 'coupon_id'], how='left')
df_x.shape
df_x = df_x[df_x.start_date > df_x.date]
df_x.shape

cust_coup_hist_trans = df_x.groupby(['customer_id', 'coupon_id', 'start_date']).agg({
            'item_id': ['count'],
            'selling_price': ['sum', 'mean', 'std'],
            'other_discount': ['sum', 'mean'],
            'coupon_discount': ['sum', 'mean'], 
            'coupon_applied': ['sum', 'mean'], 
})

cust_coup_hist_trans.columns = ['_'.join(col).strip('_') for col in cust_coup_hist_trans.columns.values]
cust_coup_hist_trans = cust_coup_hist_trans.reset_index()
cust_coup_hist_trans.shape
cust_coup_hist_trans.head()

(365092, 17)

(235344, 17)

(24694, 13)

,customer_id,coupon_id,start_date,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean
0,1,24,2013-05-19,76,8487.78,111.681316,29.993760,1086.71,14.298816,130.03,1.710921,8.0,0.105263
1,1,30,2013-08-10,110,9842.82,89.480182,34.782438,1690.25,15.365909,74.79,0.679909,5.0,0.045455
2,1,32,2013-08-10,42,5529.96,131.665714,81.467997,753.00,17.928571,324.10,7.716667,14.0,0.333333
3,1,38,2013-09-16,22,1001.91,45.541364,12.109585,380.75,17.306818,0.00,0.000000,0.0,0.000000
4,1,189,2013-08-10,2,213.72,106.860000,25.187144,127.52,63.760000,0.00,0.000000,0.0,0.000000


In [20]:
cust_coup_hist_trans.to_csv('cust_coup_hist_trans.csv', index=False)

In [21]:
cust_coup_daily = coup_trans.groupby([ 'customer_id', 'coupon_id', 'date']).agg({
    'item_id': 'count',
    'quantity': 'sum',
    'selling_price': 'sum',
    'other_discount': 'sum',
    'coupon_discount': 'sum',
    'coupon_applied': 'sum'
})
cust_coup_daily = cust_coup_daily.reset_index()
cust_coup_daily = cust_coup_daily.sort_values(by=['customer_id', 'coupon_id', 'date'])

cust_coup_daily['selling_price_ewm'] = cust_coup_daily.groupby(['customer_id', 'coupon_id'])['selling_price'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()
cust_coup_daily['coupon_discount_ewm'] = cust_coup_daily.groupby(['customer_id', 'coupon_id'])['coupon_discount'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()
cust_coup_daily['coupon_applied_ewm'] = cust_coup_daily.groupby(['customer_id', 'coupon_id'])['coupon_applied'].apply(
    lambda x: x.ewm(com=0.5).mean()).tolist()

cust_coup_daily.shape
cust_coup_daily.head()

df_x = df_[['customer_id', 'coupon_id', 'start_date']].drop_duplicates().merge(cust_coup_daily, on=['customer_id', 'coupon_id'])
df_x = df_x[df_x.start_date > df_x.date]
df_x = df_x.sort_values(by=['customer_id', 'coupon_id', 'start_date', 'date'])
df_x.shape

cust_coup_hist_trans_daily = df_x.groupby(['customer_id', 'coupon_id', 'start_date']).agg({
    'date': ['count'],
    'selling_price': ['mean', 'std', 'last'],
    'other_discount': ['mean', 'std', 'last'],
    'coupon_discount': ['mean', 'std', 'last'],
    'coupon_applied': ['mean', 'std', 'last'],
    'selling_price_ewm': ['mean', 'std'],
    'coupon_discount_ewm': ['mean', 'std'],
    'coupon_applied_ewm': ['mean', 'std'],
})

cust_coup_hist_trans_daily.columns = ['_'.join(col).strip('_') for col in cust_coup_hist_trans_daily.columns.values]
cust_coup_hist_trans_daily = cust_coup_hist_trans_daily.reset_index()
cust_coup_hist_trans_daily.shape
cust_coup_hist_trans_daily.head()

(1164091, 12)

,customer_id,coupon_id,date,item_id,quantity,selling_price,other_discount,coupon_discount,coupon_applied,selling_price_ewm,coupon_discount_ewm,coupon_applied_ewm
0,1.0,1,2012-10-18,1,1.0,71.24,35.26,0.0,0.0,71.240000,0.0,0.0
1,1.0,1,2013-03-12,1,1.0,71.24,35.26,0.0,0.0,71.240000,0.0,0.0
2,1.0,6,2012-02-21,1,1.0,35.62,17.45,0.0,0.0,35.620000,0.0,0.0
3,1.0,6,2012-03-29,1,1.0,35.26,17.81,0.0,0.0,35.350000,0.0,0.0
4,1.0,6,2012-05-17,1,1.0,44.52,12.11,0.0,0.0,41.698462,0.0,0.0


(131723, 13)

(24694, 22)

,customer_id,coupon_id,start_date,date_count,selling_price_mean,selling_price_std,selling_price_last,other_discount_mean,other_discount_std,other_discount_last,coupon_discount_mean,coupon_discount_std,coupon_discount_last,coupon_applied_mean,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std
0,1,24,2013-05-19,33,257.205455,117.976874,248.62,32.930606,49.507987,0.00,3.940303,8.631704,0.0,0.242424,0.501890,0.0,258.167855,84.349758,4.267401,7.110299,0.250083,0.345649
1,1,30,2013-08-10,43,228.902791,113.028578,166.70,39.308140,44.304940,5.34,1.739302,6.076651,0.0,0.116279,0.324353,0.0,229.945091,79.764926,1.739517,4.237990,0.116303,0.232132
2,1,32,2013-08-10,23,240.433043,187.939520,195.55,32.739130,39.990996,21.02,14.091304,20.210501,0.0,0.608696,0.940944,0.0,243.344702,114.306735,14.067949,15.099110,0.608200,0.673791
3,1,38,2013-09-16,20,50.095500,19.941819,38.11,19.037500,13.210864,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,50.068695,13.237522,0.000000,0.000000,0.000000,0.000000
4,1,189,2013-08-10,2,106.860000,25.187144,89.05,63.760000,25.187144,81.57,0.000000,0.000000,0.0,0.000000,0.000000,0.0,111.312500,18.890358,0.000000,0.000000,0.000000,0.000000


In [22]:
cust_coup_hist_trans_daily.to_csv('cust_coup_hist_trans_daily.csv', index=False)

#### coupon brand

In [23]:
coupon_items = coupon_item_mapping.merge(item_data, on='item_id')
coupon_items.shape
coupon_items.head()

(92663, 5)

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,6,37,56,Local,Grocery
2,22,37,56,Local,Grocery
3,31,37,56,Local,Grocery
4,107,75,56,Local,Grocery


In [24]:
coupon_brands = coupon_items[['coupon_id', 'brand']].drop_duplicates()
coupon_brands.shape
coupon_brands.head()

(7876, 2)

,coupon_id,brand
0,105,56
1,6,56
2,22,56
3,31,56
4,107,56


In [25]:
coupon_brand_trans = coupon_brands.merge(cust_transactions, on='brand', how='left')
coupon_brand_trans.shape

(110493073, 12)

In [26]:
coupon_brand_trans.head()

,coupon_id,brand,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand_type,category,coupon_applied
0,105,56,2012-01-02,1501,26830,1,35.26,10.69,0.0,Local,Natural Products,0
1,105,56,2012-01-02,1501,54253,1,53.43,13.89,0.0,Local,Natural Products,0
2,105,56,2012-01-02,67,19560,2,106.86,2.85,0.0,Local,Grocery,0
3,105,56,2012-01-02,67,21653,1,44.52,12.11,0.0,Local,Grocery,0
4,105,56,2012-01-02,751,34047,1,53.43,0.00,0.0,Local,Grocery,0


In [27]:
df_x = df_.merge(coupon_brand_trans, on=['customer_id', 'coupon_id'], how='left')
df_x.shape
df_x = df_x[df_x.start_date > df_x.date]
df_x.shape

(7901633, 17)

(6622738, 17)

In [28]:
coup_brand_hist_trans= df_x.groupby(['coupon_id', 'start_date']).agg({
            'item_id': ['count'],
            'selling_price': ['sum', 'mean', 'std'],
            'other_discount': ['sum', 'mean'],
            'coupon_discount': ['sum', 'mean'], 
            'coupon_applied': ['sum', 'mean'], 
})

coup_brand_hist_trans.columns = ['_'.join(col).strip('_') for col in coup_brand_hist_trans.columns.values]
coup_brand_hist_trans = coup_brand_hist_trans.reset_index()
coup_brand_hist_trans.shape
coup_brand_hist_trans.head()

(1299, 12)

,coupon_id,start_date,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean
0,1,2012-08-12,45,6348.50,141.077778,74.767759,2226.25,49.472222,19.59,0.435333,1.0,0.022222
1,2,2012-08-12,7,1279.82,182.831429,71.559057,256.46,36.637143,0.00,0.000000,0.0,0.000000
2,3,2012-10-08,1142,116454.39,101.974072,51.133843,25203.88,22.069947,373.97,0.327469,15.0,0.013135
3,4,2012-11-19,190,14803.70,77.914211,48.466975,4330.97,22.794579,17.81,0.093737,1.0,0.005263
4,5,2012-11-19,2,408.91,204.455000,88.155002,245.78,122.890000,0.00,0.000000,0.0,0.000000


In [29]:
coup_brand_hist_trans.to_csv('coup_brand_hist_trans.csv', index=False)

In [30]:
cust_brand_hist_trans= df_x.groupby(['customer_id', 'start_date']).agg({
            'item_id': ['count'],
            'selling_price': ['sum', 'mean', 'std'],
            'other_discount': ['sum', 'mean'],
            'coupon_discount': ['sum', 'mean'], 
            'coupon_applied': ['sum', 'mean'], 
})

cust_brand_hist_trans.columns = ['_'.join(col).strip('_') for col in cust_brand_hist_trans.columns.values]
cust_brand_hist_trans = cust_brand_hist_trans.reset_index()
cust_brand_hist_trans.shape
cust_brand_hist_trans.head()

(6929, 12)

,customer_id,start_date,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean
0,1,2012-10-08,190,18557.88,97.673053,71.501037,6022.00,31.694737,277.81,1.462158,9.0,0.047368
1,1,2013-02-16,463,40831.36,88.188683,42.611269,11350.91,24.516004,320.58,0.692397,11.0,0.023758
2,1,2013-04-22,360,29895.96,83.044333,40.027334,7283.11,20.230861,96.15,0.267083,7.0,0.019444
3,1,2013-05-19,661,57123.14,86.419274,38.743368,15387.30,23.278820,130.03,0.196717,8.0,0.012103
4,1,2013-08-10,1194,103186.17,86.420578,42.526696,25163.26,21.074757,914.99,0.766323,35.0,0.029313


In [31]:
cust_brand_hist_trans.to_csv('cust_brand_hist_trans.csv', index=False)

In [32]:
cust_coup_brand_hist_trans= df_x.groupby(['customer_id', 'coupon_id', 'start_date']).agg({
            'item_id': ['count'],
            'selling_price': ['sum', 'mean', 'std'],
            'other_discount': ['sum', 'mean'],
            'coupon_discount': ['sum', 'mean'], 
            'coupon_applied': ['sum', 'mean'], 
})
cust_coup_brand_hist_trans.columns = ['_'.join(col).strip('_') for col in cust_coup_brand_hist_trans.columns.values]
cust_coup_brand_hist_trans = cust_coup_brand_hist_trans.reset_index()
cust_coup_brand_hist_trans.shape
cust_coup_brand_hist_trans.head()

(75495, 13)

,customer_id,coupon_id,start_date,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean
0,1,3,2012-10-08,18,1659.17,92.176111,36.006515,382.92,21.273333,16.02,0.890000,2.0,0.111111
1,1,24,2013-05-19,172,16276.07,94.628314,38.498848,3328.61,19.352384,130.03,0.755988,8.0,0.046512
2,1,30,2013-08-10,289,22690.77,78.514775,38.254924,5249.69,18.165017,81.91,0.283426,6.0,0.020761
3,1,32,2013-08-10,243,24433.79,100.550576,53.058817,4313.54,17.751193,781.44,3.215802,26.0,0.106996
4,1,38,2013-09-16,57,2542.07,44.597719,15.372912,571.66,10.029123,7.12,0.124912,1.0,0.017544


In [33]:
cust_coup_brand_hist_trans.to_csv('cust_coup_brand_hist_trans.csv', index=False)